In [11]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname = "C:\Windows\Fonts\malgun.ttf").get_name()
rc('font', family=font_name)
import seaborn as sns
from scipy.optimize import minimize
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib inline

In [12]:
asset_code = ['005930', '035720', '005490', '068270', '018260', '051900', '096770', '036570', '010130', '021240', '035250']
asset_number = len(asset_code)

In [13]:
stock_listing = fdr.StockListing('KOSPI')
asset_list = []

for code in asset_code :
    asset_list.append(stock_listing[stock_listing['Symbol'] == code])

asset_info = pd.concat(asset_list)

In [14]:
asset_data = []

for code in asset_code :
    asset_close = fdr.DataReader(code,'2017')[['Close']]
    asset_data.append(asset_close)

asset_price = pd.concat(asset_data, axis = 1)
asset_price.columns = asset_info['Name'].values

In [15]:
data = asset_price.pct_change()

In [16]:
ratio = 0.3

In [17]:
risk_type = 'Risk preference'

In [25]:
def goodan_optimizer(value, asset_number, data, ratio, risk_type) :
    
    expected_return = np.array(data.mean()*252)
    covariance_martix = data.cov()*252

    def objective(weights):
        return np.sqrt(np.dot(weights.T, np.dot(covariance_martix, weights)))

    def constraint1(weights):
        return sum(weights)-1

    def constraint2(weights):
        return weights[-1] - ratio

    w0 = np.array([(1/asset_number) for i in range(asset_number)])
    bounds = [[0,1] for i in range(asset_number)]
    constraints = [{'type':'eq','fun':constraint1}, {'type':'eq','fun':constraint2}]

    solution = minimize(objective,w0,method='SLSQP', bounds = bounds, constraints = constraints)
    return_gmv = (expected_return*solution.x).sum()
    
    def investment_tendency(weight) :
        return np.sqrt(np.dot(weight.T, np.dot(covariance_martix, weight)))

    basic_var = np.array([(1/asset_number) for i in range(asset_number)])
    bnds = [[0,1] for i in range(asset_number)]
    cons = [{'type':'eq','fun':constraint1}]

    ef_return = [return_gmv, return_gmv+0.05, return_gmv+0.10]
    investment_tendency_rank = ['Risk aversion', 'Risk neutrality', 'Risk preference']

    if risk_type == investment_tendency_rank[0] :
        r = ef_return[0]

    elif risk_type == investment_tendency_rank[1] :
        r = ef_return[1]

    elif risk_type == investment_tendency_rank[2] :
        r = ef_return[2]

    con1 = {'type':'eq','fun': lambda x : np.sum(x) - 1}
    con2 = {'type':'eq','fun': lambda x : (expected_return*x).sum() - r}
    con3 = {'type':'eq','fun': lambda x : x[-1] - ratio}
    cons = [con1,con2,con3]
    solution_risk_type = minimize(investment_tendency, basic_var, method = 'SLSQP', bounds = bnds, constraints = cons)

    return value * solution_risk_type.x.round(3), solution_risk_type.x.round(3)

In [26]:
goodan_optimizer(8500000, asset_number, data, ratio, risk_type)

(array([ 748000., 1606500.,       0.,  731000.,       0., 1079500.,
              0., 1776500.,       0.,       0., 2550000.]),
 array([0.088, 0.189, 0.   , 0.086, 0.   , 0.127, 0.   , 0.209, 0.   ,
        0.   , 0.3  ]))